In [1]:
import pandas as pd
import torch
import torch.utils.data as data
import numpy as np
import torchvision.transforms as transforms


class CK(data.Dataset):
    def __init__(self, mode):
        self.mode = mode
        self.train_set = []
        self.validation_set = []
        self.test_set = []

        
        db = pd.read_csv('./dataset/ckextended.csv', delimiter=',')

        #Carico, a partire dal file csv, le immagini da utilizzare per addestrare la rete
        db_train = db[db['Usage'] == 'Training']
        db_train.drop('Usage', axis=1)

        db_validation = db[db['Usage'] == 'PrivateTest']
        db_validation.drop('Usage', axis=1)

        db_test = db[db['Usage'] == 'PublicTest']
        db_test.drop('Usage', axis=1)

        # Creo i vettori che conterranno il training e testing set
        self.label_train = [label for label in db_train['emotion']]
        self.label_validation = [label for label in db_validation['emotion']]
        self.label_test = [label for label in db_test['emotion']]

        for img in db_train['pixels']:
            tmp = []
            for pixel in img.split(' '):
                tmp.append(int(pixel))
            self.train_set.append(tmp)

        for img in db_validation['pixels']:
            tmp = []
            for pixel in img.split(' '):
                tmp.append(int(pixel))
            self.validation_set.append(tmp)

        for img in db_test['pixels']:
            tmp = []
            for pixel in img.split(' '):
                tmp.append(int(pixel))
            self.test_set.append(tmp)
    
    
    def __getitem__(self, index):
        
        if self.mode == 'Training':
            img, label = self.train_set[index], self.label_train[index]
        
        if self.mode == 'Testing':
            img, label = self.test_set[index], self.label_test[index]
            
        if self.mode == 'Validation':
            img, label = self.validation_set[index], self.validation_set[index]

        img = np.array(img, dtype=float)
        img = img.reshape((1, 48, 48))
        img = np.concatenate((img, img, img), axis=0)
        img = torch.tensor(img, dtype=torch.float)
        
        return img, label

    def __len__(self):
        if self.mode == 'Training':
            return len(self.train_set)
        if self.mode == 'Testing':
            return len(self.test_set)
        if self.mode == 'Validation':
            return len(self.validation_set)

In [2]:
import torch

train = CK('Training')
trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=1)

test = CK('Testing')
testloader = torch.utils.data.DataLoader(test, batch_size=5, shuffle=False, num_workers=1)


In [3]:
from VGG import VGG19
import torch.nn as nn
import torch.optim as optim
net = VGG19()

if torch.cuda.is_available():
    net.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


In [4]:
def train(epoch):
    print("Epoca:" + str(epoch))
    
    net.train()
    
    for batch_id, (inputs, labels) in enumerate(trainloader):
        
        inputs, labels = inputs.cuda(), labels.cuda()
        print(inputs.shape)
    
        # Azzeramento dei gradienti
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs)
        
        # Calcolo della loss
        loss = criterion(outputs, labels)
        
        # Calcolo dei gradienti
        loss.backward()
        
        # Aggiornamento dei parametri
        optimizer.step()
         

In [5]:
for epoch in range(0, 5):
    train(epoch)
    test(epoch)

Epoca:0
torch.Size([128, 3, 48, 48])


../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [12,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
